In [1]:
!pip install split_folders

In [2]:
import splitfolders

In [3]:
input_folder = "/kaggle/input/new-image-classification-dataset-aml/IMAGE"
output = "/kaggle/working/processed"
splitfolders.ratio(input_folder, output, seed=42, ratio=(0.6, 0.2, 0.2))

Copying files: 1794 files [00:11, 154.63 files/s]


In [4]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

print("All Modules Imported Successfully")

2024-04-29 11:54:04.378072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 11:54:04.378226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 11:54:04.511551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


All Modules Imported Successfully


In [5]:
# Define image dimensions
img_height, img_width = (224, 224)
batch_size = 32

In [6]:
train_data_dir = r"/kaggle/working/processed/train"
valid_data_dir = r"/kaggle/working/processed/val"
test_data_dir = r"/kaggle/working/processed/test"

In [7]:
# Data preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [8]:
#Set as Training Data

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 858 images belonging to 26 classes.


In [9]:
#Set as Validation Data

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 52 images belonging to 26 classes.


In [10]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation'
)

Found 78 images belonging to 26 classes.


In [11]:
#Building the Base Model

base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [12]:
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
#Freezing the Base Model Layers

for layer in base_model.layers:
    layer.trainable = False

In [14]:
#Compiling the Model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [15]:
#Fitting the Model

model.fit(train_generator, epochs = 10)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.3769 - loss: 2.6719
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.9046 - loss: 0.3533
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.9608 - loss: 0.1760
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9927 - loss: 0.0650
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.9978 - loss: 0.0481
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.9988 - loss: 0.0230
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9962 - loss: 0.0237
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 1.0000 - loss: 0.0179
Epoch 9/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.9976 - loss: 0.0225
Epoch 10/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 93s 3s/step - accuracy: 0.9963 - loss: 0.0203


In [16]:
#Checking the Accuracy

test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("\nTest Accuracy: ",test_acc*100)

78/78 - 14s - 175ms/step - accuracy: 1.0000 - loss: 0.0110

Test Accuracy:  100.0
